### 직방의 원룸 매물정보 수집
- 절차
    -동이름> 위도,경도 수집
    -위도,경도>geohash(영역) 변환
    -geohash>매물 아이디
    -매물 아이디> 매물정보

In [5]:
import requests
import pandas as pd
import geohash2

In [2]:
# 1.동이름 >위도, 경도

In [11]:
addr = "서초동"
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
response = requests.get(url)
data = response.json()['items'][0]
lat, lng = data['lat'],data['lng']
lat,lng

(37.49008560180664, 127.01953125)

In [1]:
# 2. 위도 경두>geohash

In [13]:
geohash = geohash2.encode(lat,lng,precision=5)
geohash

'wydm6'

In [16]:
url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'

response = requests.get(url)
response

<Response [200]>

In [ ]:
# 3. geohash > 매물 아이디

In [17]:
items = response.json()['items']
len(items),items[:2]

(2344,
 [{'lat': 37.485141261794524, 'lng': 127.01296495741643, 'item_id': 32819635},
  {'lat': 37.485627384450254, 'lng': 127.0136283391101, 'item_id': 32838798}])

In [18]:
ids = [item["item_id"] for item in items]
ids[:5]

[32819635, 32838798, 32872608, 32838700, 32767020]

In [ ]:
# 4. 매물 아이디 > 매물 정보

In [20]:
url = 'https://apis.zigbang.com/v2/items/list'
params = {
    'domain':'zigbang',
    'withCoalition': 'true',
    'item_ids': ids[:900]
}
response = requests.post(url,params)
response

<Response [200]>

In [25]:
items = response.json()["items"]
columns = ['item_id','sales_type','deposit','rent','size_m2','address1','manage_cost']
df = pd.DataFrame(items)[columns]
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
898,32844154,월세,100,100,29.75,서울시 강남구 역삼동,9
899,32656329,전세,24000,0,59.50,서울시 강남구 역삼동,10


In [30]:
#생략값 뺴고 출력
#max row, max column 설정
pd.options.display.max_columns,pd.options.display.max_rows

(60, 60)

In [27]:
pd.options.display.max_columns=60

In [28]:
pd.DataFrame(items)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
0,None,32819635,https://ic.zigbang.com/ic/items/32819635/1.jpg,전세,전세,16500,0,39.89,"{'m2': 39.89, 'p': '12.1'}","{'m2': 20.82, 'p': '6.3'}",None,None,중,중,20,★전입신고O★10월14일이후★채광없음★일반전세자금대출가능★,None,01,서초구 서초동,"{'lat': 37.48514381187695, 'lng': 127.01297854...",False,True,오피스텔,[],서울시 서초구 서초동,None,None,7.5,2022-07-30T13:43:13+09:00,False
1,None,32838798,https://ic.zigbang.com/ic/items/32838798/1.jpg,월세,월세,1000,68,37.56,"{'m2': 37.56, 'p': '11.4'}","{'m2': 19.6, 'p': '5.9'}",None,None,중,중,20,★전입신고불가★채광없음★9/2이후★보안굿★신축급★주차가능★,None,01,서초구 서초동,"{'lat': 37.48566302280698, 'lng': 127.01360580...",False,True,오피스텔,[추천],서울시 서초구 서초동,None,None,7.5,2022-08-03T14:37:56+09:00,True
2,None,32872608,https://ic.zigbang.com/ic/items/32872608/1.jpg,월세,월세,1000,70,37.56,"{'m2': 37.56, 'p': '11.4'}","{'m2': 19.6, 'p': '5.9'}",None,None,저,저,20,★전입신고가능★10월1일이후★채광없음★주차가능★관리비저렴★,None,01,서초구 서초동,"{'lat': 37.48513877563711, 'lng': 127.01361606...",False,True,오피스텔,[추천],서울시 서초구 서초동,None,None,7.5,2022-08-03T19:08:33+09:00,True
3,None,32838700,https://ic.zigbang.com/ic/items/32838700/1.jpg,전세,전세,50400,0,66.12,"{'m2': 66.12, 'p': '20'}","{'m2': 52.56, 'p': '15.9'}",None,None,4,4,6,★서초동 실사용20평 3년차 신축3룸(고급주택가/주차좋음),None,05,서초구 서초동,"{'lat': 37.48516264325736, 'lng': 127.00286814...",False,True,빌라,[추천],서울시 서초구 서초동,None,None,9,2022-08-01T15:05:10+09:00,False
4,None,32767020,https://ic.zigbang.com/ic/items/32767020/1.jpg,월세,월세,20000,90,60.12,"{'m2': 60.12, 'p': '18.2'}","{'m2': 59.61, 'p': '18'}",None,None,1,1,3,🐤드디어 이 집 나왔습니다 !!!!･ᴗ･🐤,None,04,서초구 서초동,"{'lat': 37.48565191121936, 'lng': 127.03058800...",False,True,빌라,[],서울시 서초구 서초동,None,None,5,2022-08-03T17:05:14+09:00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,None,32850137,https://ic.zigbang.com/ic/items/32850137/1.jpg,월세,월세,300,260,32.91,"{'m2': 32.91, 'p': '10'}","{'m2': 27.22, 'p': '8.2'}",None,None,4,4,6,📙2룸 신축빌라📒조명맛집📗주차편리📘,None,04,강남구 역삼동,"{'lat': 37.5020916971116, 'lng': 127.034354038...",False,True,빌라,[추천],서울시 강남구 역삼동,None,None,10,2022-08-02T11:59:56+09:00,False
896,None,32852414,https://ic.zigbang.com/ic/items/32852414/1.jpg,월세,월세,130,250,33.44,"{'m2': 33.44, 'p': '10.1'}","{'m2': 27.65, 'p': '8.4'}",None,None,3,3,6,✅✅ 100% 🌞 실매물 ✅✅ 풀옵션 투룸,None,04,강남구 역삼동,"{'lat': 37.50207639752684, 'lng': 127.03434324...",False,True,빌라,[추천],서울시 강남구 역삼동,None,None,10,2022-08-02T15:04:01+09:00,True
897,None,32809154,https://ic.zigbang.com/ic/items/32809154/1.jpg,월세,월세,120,70,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}",None,None,3,3,4,🔴⭕무조건❗최저가⭕🔴 역삼역 바로 코앞에~,None,01,강남구 역삼동,"{'lat': 37.50211314475644, 'lng': 127.03407136...",False,True,원룸,[추천],서울시 강남구 역삼동,None,None,9,2022-07-29T15:01:31+09:00,False
898,None,32844154,https://ic.zigbang.com/ic/items/32844154/1.jpg,월세,월세,100,100,29.75,"{'m2': 29.75, 'p': '9'}","{'m2': 29.75, 'p': '9'}",None,None,2,2,2,🟥🟦단기💖맛집🟦🟥 리모델링 주거/업무 겸용 원룸!,None,01,강남구 역삼동,"{'lat': 37.50167352690325, 'lng': 127.02889363...",False,True,원룸,[추천],서울시 강남구 역삼동,None,None,9,2022-08-01T18:43:35+09:00,False


In [31]:
def oneroom(addr):
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    data = response.json()['items'][0]
    lat, lng = data['lat'],data['lng']
    geohash = geohash2.encode(lat,lng,precision=5)
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    items = response.json()['items']
    ids = [item["item_id"] for item in items]
    url = 'https://apis.zigbang.com/v2/items/list'
    params = {
        'domain':'zigbang',
        'withCoalition': 'true',
        'item_ids': ids[:900]
    }
    response = requests.post(url,params)
    items = response.json()["items"]
    columns = ['item_id','sales_type','deposit','rent','size_m2','address1','manage_cost']
    return pd.DataFrame(items)[columns]

In [35]:
addr='동작구 사당동'
df = oneroom(addr)
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
821,32732194,전세,19000,0,26.45,서울시 서초구 방배동,15
822,32590850,월세,500,30,19.83,서울시 동작구 사당동,0


In [36]:
df_filtered=df[df['address1'].str.contains(addr)].reset_index(drop=True)
df_filtered

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32502575,전세,23000,0,39.00,서울시 동작구 사당동,10
1,32563188,전세,23000,0,69.42,서울시 동작구 사당동,10
2,32612287,전세,16000,0,23.14,서울시 동작구 사당동,10
3,32678164,전세,23000,0,33.06,서울시 동작구 사당동,10
4,32678697,전세,13000,0,16.53,서울시 동작구 사당동,10
...,...,...,...,...,...,...,...
300,32827257,월세,1000,40,19.83,서울시 동작구 사당동,10
301,32864863,전세,10000,0,19.83,서울시 동작구 사당동,10
302,32655484,월세,500,35,19.83,서울시 동작구 사당동,0
303,32859283,월세,500,38,45.59,서울시 동작구 사당동,7
